In [8]:
using Distributed

In [9]:
addprocs(3)

3-element Array{Int64,1}:
 2
 3
 4

In [23]:
@everywhere using LinearAlgebra
@distributed (+) for i in 1:10
    @show svd(randn(10,10)).S[1]
end

      From worker 2:	(svd(randn(10, 10))).S[1] = 6.464464619426728
      From worker 2:	(svd(randn(10, 10))).S[1] = 5.251658463470373
      From worker 2:	(svd(randn(10, 10))).S[1] = 5.1529160408113555
      From worker 2:	(svd(randn(10, 10))).S[1] = 5.747751299687305
      From worker 4:	(svd(randn(10, 10))).S[1] = 6.568148862926666
      From worker 4:	(svd(randn(10, 10))).S[1] = 5.143129111255482
      From worker 4:	(svd(randn(10, 10))).S[1] = 5.58030688695713
      From worker 3:	(svd(randn(10, 10))).S[1] = 6.029567427723876
      From worker 3:	(svd(randn(10, 10))).S[1] = 5.131227949108335
      From worker 3:	(svd(randn(10, 10))).S[1] = 5.0091430348431025


56.07831369621035

In [5]:
#= Julia code for launching jobs on the slurm cluster.

This code is expected to be run from an sbatch script after a module load julia command has been run.
It starts the remote processes with srun within an allocation.
If you get an error make sure to Pkg.checkout("CluterManagers").

=#
try
	using ClusterManagers
catch
    using Pkg
    Pkg.activate(".")
	Pkg.add("ClusterManagers")
	Pkg.checkout("ClusterManagers")
end


In [6]:
using ClusterManagers
# Arguments to the Slurm srun(1) command can be given as keyword
# arguments to addprocs.  The argument name and value is translated to
# a srun(1) command line argument as follows:
# 1) If the length of the argument is 1 => "-arg value",
#    e.g. t="0:1:0" => "-t 0:1:0"
# 2) If the length of the argument is > 1 => "--arg=value"
#    e.g. time="0:1:0" => "--time=0:1:0"
# 3) If the value is the empty string, it becomes a flag value,
#    e.g. exclusive="" => "--exclusive"
# 4) If the argument contains "_", they are replaced with "-",
#    e.g. mem_per_cpu=100 => "--mem-per-cpu=100"

np = 4 #
addprocs(SlurmManager(np), t="00:5:00")

UndefVarError: UndefVarError: addprocs not defined

In [ ]:
hosts = []
pids = []
println("We are all connected and ready.")
for i in workers()
	host, pid = fetch(@spawnat i (gethostname(), getpid()))
	println(host, pid)
	push!(hosts, host)
	push!(pids, pid)
end

# The Slurm resource allocation is released when all the workers have
# exited
for i in workers()
	rmprocs(i)
end